In [ ]:
import cv2
import os
import glob

base_dir = "/home/dw/ws_job_msislab/amr_project/data/video_data/20251217_video_2"

# 최상위 images 폴더 생성
root_output_dir = os.path.join(base_dir, "images")
os.makedirs(root_output_dir, exist_ok=True)

# MP4 파일 리스트 가져오기
video_list = sorted(glob.glob(os.path.join(base_dir, "color_output_*.mp4")))

print(f"[INFO] 감지된 MP4 파일 수: {len(video_list)}")

for video_path in video_list:
    video_name = os.path.basename(video_path).replace(".mp4", "")
    print(f"\n[START] {video_name} 변환 시작")

    # 영상별 폴더 생성
    output_dir = os.path.join(root_output_dir, video_name)
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"[ERROR] 비디오 열기 실패: {video_path}")
        continue

    frame_idx = 1  # 영상마다 1부터 시작

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        prefix = "20251216"
        save_name = f"{prefix}_{video_name}_{frame_idx:06d}.jpg"
        save_path = os.path.join(output_dir, save_name)
        cv2.imwrite(save_path, frame)

        frame_idx += 1

        if frame_idx % 500 == 0:
            print(f"[INFO] {video_name}: {frame_idx}장 저장 중…")

    cap.release()
    print(f"[DONE] {video_name}: 총 {frame_idx-1}장 저장")

print("\n[ALL DONE] 모든 MP4 변환 완료!")



In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 여러 원본 폴더들
origin_bases = [
    "/home/dw/ws_job_msislab/amr_project/data/image_data/for_yolo_cls/20251216_2",
    "/home/dw/ws_job_msislab/amr_project/data/image_data/for_yolo_cls/20251217",
    "/home/dw/ws_job_msislab/amr_project/data/image_data/for_yolo_cls/20251217_2",
]

target_base = "/home/dw/ws_job_msislab/amr_project/for_training/20251217_2"

train_dir = os.path.join(target_base, "train")
val_dir = os.path.join(target_base, "val")

# 기존 결과 삭제 후 다시 생성
shutil.rmtree(target_base, ignore_errors=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

classes = ["box_0", "box_1", "box_2", "box_3", "box_4", "unknown"]
IMG_EXTS = (".jpg", ".jpeg", ".png")

TEST_SIZE = 0.15
RANDOM_STATE = 25

def copy_with_prefix(src_path: str, dst_folder: str, prefix: str):
    """
    폴더명 prefix 붙여 저장해서 충돌 방지 + 출처 추적 가능
    """
    base = os.path.basename(src_path)
    name, ext = os.path.splitext(base)
    out_name = f"{prefix}__{name}{ext}"
    dst_path = os.path.join(dst_folder, out_name)

    # 혹시 같은 파일이 또 있으면 _1, _2 붙이기
    if not os.path.exists(dst_path):
        shutil.copy2(src_path, dst_path)
        return

    i = 1
    while True:
        out_name2 = f"{prefix}__{name}_{i}{ext}"
        dst_path2 = os.path.join(dst_folder, out_name2)
        if not os.path.exists(dst_path2):
            shutil.copy2(src_path, dst_path2)
            return
        i += 1


# 클래스별 전체 누적 카운트용
total_train = {c: 0 for c in classes}
total_val = {c: 0 for c in classes}

for origin_base in origin_bases:
    folder_tag = os.path.basename(origin_base.rstrip("/"))

    print(f"\n========== [ORIGIN] {origin_base} (tag={folder_tag}) ==========")

    for cls in classes:
        src_folder = os.path.join(origin_base, cls)

        if not os.path.isdir(src_folder):
            print(f"[WARN] 폴더 없음: {src_folder} (skip)")
            continue

        images = [
            os.path.join(src_folder, f)
            for f in os.listdir(src_folder)
            if f.lower().endswith(IMG_EXTS)
        ]

        print(f"[INFO] {folder_tag}/{cls}: 총 {len(images)}장")

        if len(images) == 0:
            continue

        # 너무 적으면 val이 0장이 될 수 있으니 예외 처리
        if len(images) < 2:
            train_imgs, val_imgs = images, []
        else:
            # 폴더별(=origin_base별) 85/15 유지 split
            train_imgs, val_imgs = train_test_split(
                images,
                test_size=TEST_SIZE,
                random_state=RANDOM_STATE,
                shuffle=True
            )

        train_cls_dir = os.path.join(train_dir, cls)
        val_cls_dir = os.path.join(val_dir, cls)
        os.makedirs(train_cls_dir, exist_ok=True)
        os.makedirs(val_cls_dir, exist_ok=True)

        for img in train_imgs:
            copy_with_prefix(img, train_cls_dir, prefix=folder_tag)
        for img in val_imgs:
            copy_with_prefix(img, val_cls_dir, prefix=folder_tag)

        total_train[cls] += len(train_imgs)
        total_val[cls] += len(val_imgs)

        print(f"[DONE] {folder_tag}/{cls} → train {len(train_imgs)}, val {len(val_imgs)}")

print("\n========== [SUMMARY] 전체 누적 ==========")
for cls in classes:
    print(f"{cls:10s} | train {total_train[cls]:6d} | val {total_val[cls]:6d}")

print("\n[ALL DONE] 폴더별 비율 유지(각 폴더 85/15) 후 합쳐서 완료!")


In [2]:
from ultralytics import YOLO

# -------------------------------------------------
# 모델 로드 (classification)
# -------------------------------------------------
model = YOLO("yolo11s-cls.pt")

data_path = "/home/dw/ws_job_msislab/amr_project/for_training/20251217_2"

# -------------------------------------------------
# 학습
# -------------------------------------------------
model.train(
    data=data_path,
    epochs=500,
    imgsz=224,
    batch=32,
    patience=25,
    save=True,
    save_period=0,
    seed=25,
    deterministic=True,

    optimizer="AdamW",
    lr0=1e-3,
    lrf=1e-2,
    weight_decay=5e-4,
    momentum=0.9,

    cos_lr=True,
    warmup_epochs=3,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,

    hsv_h=0.015,
    hsv_s=0.4,
    hsv_v=0.4,
    degrees=5.0,
    translate=0.05,
    scale=0.2,
    shear=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=0.0,
    mixup=0.1,
    copy_paste=0.0,

    erasing=0.0,          # ✅ Random Erasing(검은 네모) 끄기

    label_smoothing=0.05,
    dropout=0.2,

    val=True,
    split="val",
    save_json=False,

    device=0,
    workers=8,
    amp=True,
    cache=False,
    verbose=True,
)




New https://pypi.org/project/ultralytics/8.3.239 available 😃 Update with 'pip install -U ultralytics'
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.202 🚀 Python-3.10.12 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/home/dw/ws_job_msislab/amr_project/for_training/20251217_2, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=500, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ce73c32cf40>
curves: []
curves_results: []
fitness: 0.9983258843421936
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9966517686843872, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9983258843421936}
save_dir: PosixPath('/home/dw/ws_job_msislab/amr_project/src/runs/classify/train13')
speed: {'preprocess': 0.07965046093261208, 'inference': 0.31150509375420044, 'loss': 0.00017972433040215816, 'postprocess': 0.0003557812472568652}
task: 'classify'
top1: 0.9966517686843872
top5: 1.0

### obb

In [5]:
import os
import shutil

# =========================
# 경로 설정
# =========================
SRC_DIRS = [
    "/home/dw/ws_job_msislab/amr_project/data/image_data/for_yolo_obj/20251210",
    "/home/dw/ws_job_msislab/amr_project/data/image_data/for_yolo_obj/20251218",
]

DST_DIR = "/home/dw/ws_job_msislab/amr_project/for_training/20251218_obb"

# =========================
# 실행
# =========================
os.makedirs(DST_DIR, exist_ok=True)

copied = 0
skipped = 0

for src in SRC_DIRS:
    files = os.listdir(src)

    for fname in files:
        if not (fname.endswith(".jpg") or fname.endswith(".json")):
            continue

        src_path = os.path.join(src, fname)
        dst_path = os.path.join(DST_DIR, fname)

        if os.path.exists(dst_path):
            skipped += 1
            continue

        shutil.copy(src_path, dst_path)
        copied += 1

print("===================================")
print(f"✅ 복사 완료")
print(f" - 복사된 파일 : {copied}")
print(f" - 이미 존재해서 스킵 : {skipped}")
print(f" - 대상 폴더 : {DST_DIR}")
print("===================================")


✅ 복사 완료
 - 복사된 파일 : 304
 - 이미 존재해서 스킵 : 0
 - 대상 폴더 : /home/dw/ws_job_msislab/amr_project/for_training/20251218_obb


In [6]:
import json
import glob
import os

# 검사할 폴더
DIR = "/home/dw/ws_job_msislab/amr_project/for_training/20251218_obb"

# 폴더 내 모든 JSON 파일 로드
json_files = sorted(glob.glob(os.path.join(DIR, "*.json")))

print(f"\n===== Polygon 점 개수 검사 (YOLO-OBB용) =====\n")
print(f"총 JSON 파일: {len(json_files)}개\n")

if not json_files:
    print("❌ JSON 파일이 없습니다.")
    exit()

for json_path in json_files:
    print(f"\n📌 파일: {os.path.basename(json_path)}")

    try:
        with open(json_path, "r") as f:
            data = json.load(f)
    except Exception as e:
        print(f"   ❌ JSON 파일 열기 실패: {e}")
        continue

    shapes = data.get("shapes", [])

    if not shapes:
        print("   ❌ shapes 없음 (라벨링 데이터 없음)")
        continue

    for idx, s in enumerate(shapes):
        if s.get("shape_type") != "polygon":
            print(f"   [{idx}] ❌ polygon 아님 → {s.get('shape_type')}")
            continue

        pts = s.get("points", [])
        n = len(pts)

        if n == 4:
            print(f"   [{idx}] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)")
        else:
            print(f"   [{idx}] ❌ {n}점 polygon → YOLO-OBB 불가 (반드시 4점 필요)")



===== Polygon 점 개수 검사 (YOLO-OBB용) =====

총 JSON 파일: 127개


📌 파일: color_output_1_000370.json
   [0] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [1] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [2] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)

📌 파일: color_output_1_000377.json
   [0] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [1] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [2] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)

📌 파일: color_output_1_000499.json
   [0] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [1] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [2] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)

📌 파일: color_output_1_000658.json
   [0] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [1] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [2] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)

📌 파일: color_output_1_000661.json
   [0] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [1] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [2] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)

📌 파일: color_output_1_000668.json
   [0] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [1] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)
   [2] ✔ OK: 4점 polygon (YOLO-OBB 학습 가능)

In [7]:
import json
import os
from glob import glob
import numpy as np

# 변환할 폴더
BASE = "/home/dw/ws_job_msislab/amr_project/for_training/20251218_obb"

json_files = sorted(glob(os.path.join(BASE, "*.json")))

print("\n===== JSON → YOLO11-OBB 변환 시작 =====\n")


def sort_polygon_points(pts):
    """
    4개 점을 중심 기준으로 시계/반시계 순서로 정렬.
    YOLO OBB 학습을 위한 필수 단계.
    """

    pts = np.array(pts)

    # 중심점
    cx = np.mean(pts[:, 0])
    cy = np.mean(pts[:, 1])

    # 각도 계산
    angles = np.arctan2(pts[:, 1] - cy, pts[:, 0] - cx)

    # 각도 기준 정렬 (반시계 방향)
    sorted_idx = np.argsort(angles)

    return pts[sorted_idx].tolist()



for json_path in json_files:
    with open(json_path, "r") as f:
        data = json.load(f)

    img_w = data["imageWidth"]
    img_h = data["imageHeight"]

    shapes = data.get("shapes", [])

    # 출력 txt 경로
    txt_path = json_path.replace(".json", ".txt")
    lines = []

    for s in shapes:
        if s.get("shape_type") != "polygon":
            continue

        pts = s.get("points", [])

        if len(pts) != 4:
            print(f"❌ {json_path} — polygon 점 {len(pts)}개 → 4점 필수")
            continue

        # 점 순서 정렬
        pts_sorted = sort_polygon_points(pts)

        # YOLO 정규화 좌표
        yolo_pts = []
        for (x, y) in pts_sorted:
            yolo_pts.append(x / img_w)
            yolo_pts.append(y / img_h)

        # class_id = 0
        line = "0 " + " ".join([f"{v:.6f}" for v in yolo_pts])
        lines.append(line)

    # txt 파일 저장
    with open(txt_path, "w") as f:
        for line in lines:
            f.write(line + "\n")

    print(f"[OK] {os.path.basename(txt_path)} 생성됨")

print("\n===== 변환 완료 =====\n")




===== JSON → YOLO11-OBB 변환 시작 =====

[OK] color_output_1_000370.txt 생성됨
[OK] color_output_1_000377.txt 생성됨
[OK] color_output_1_000499.txt 생성됨
[OK] color_output_1_000658.txt 생성됨
[OK] color_output_1_000661.txt 생성됨
[OK] color_output_1_000668.txt 생성됨
[OK] color_output_1_000671.txt 생성됨
[OK] color_output_1_000673.txt 생성됨
[OK] color_output_1_000677.txt 생성됨
[OK] color_output_1_000685.txt 생성됨
[OK] color_output_1_000693.txt 생성됨
[OK] color_output_1_000697.txt 생성됨
[OK] color_output_1_000700.txt 생성됨
[OK] color_output_1_000704.txt 생성됨
[OK] color_output_1_000708.txt 생성됨
[OK] color_output_1_000715.txt 생성됨
[OK] color_output_1_000718.txt 생성됨
[OK] color_output_1_000721.txt 생성됨
[OK] color_output_1_000725.txt 생성됨
[OK] color_output_1_000726.txt 생성됨
[OK] color_output_1_000729.txt 생성됨
[OK] color_output_1_000745.txt 생성됨
[OK] color_output_1_000772.txt 생성됨
[OK] color_output_1_000894.txt 생성됨
[OK] color_output_1_000976.txt 생성됨
[OK] color_output_1_001090.txt 생성됨
[OK] color_output_1_001105.txt 생성됨
[OK] color_output

In [8]:
import os
import shutil
from glob import glob
import random
import yaml

BASE = "/home/dw/ws_job_msislab/amr_project/for_training/20251218_obb"

# 1) train/val/images/labels 디렉토리 생성
train_img_dir = os.path.join(BASE, "train/images")
train_lbl_dir = os.path.join(BASE, "train/labels")
val_img_dir   = os.path.join(BASE, "val/images")
val_lbl_dir   = os.path.join(BASE, "val/labels")

for d in [train_img_dir, train_lbl_dir, val_img_dir, val_lbl_dir]:
    os.makedirs(d, exist_ok=True)


print("\n===== Step 1: JSON 파일 삭제 =====")
json_files = glob(os.path.join(BASE, "*.json"))
for jf in json_files:
    os.remove(jf)
    print(f"삭제됨 → {os.path.basename(jf)}")


print("\n===== Step 2: 이미지 + 라벨 파일 수집 =====")
images = sorted(glob(os.path.join(BASE, "*.jpg")))
labels = sorted(glob(os.path.join(BASE, "*.txt")))

print(f"이미지: {len(images)}개")
print(f"라벨: {len(labels)}개")

assert len(images) == len(labels), "\n❌ 이미지/라벨 개수가 다릅니다!"


# 이미지/라벨 쌍으로 묶기
pairs = list(zip(images, labels))
random.shuffle(pairs)

val_ratio = 0.2
val_count = int(len(pairs) * val_ratio)

val_set = pairs[:val_count]
train_set = pairs[val_count:]

print(f"Train: {len(train_set)}개")
print(f"Val:   {len(val_set)}개")


def move_pairs(pairs, img_dest, lbl_dest):
    for img, lbl in pairs:
        # 파일 이동 (복사 X)
        shutil.move(img, os.path.join(img_dest, os.path.basename(img)))
        shutil.move(lbl, os.path.join(lbl_dest, os.path.basename(lbl)))


print("\n===== Step 3: 파일 이동 (train/val) =====")
move_pairs(train_set, train_img_dir, train_lbl_dir)
move_pairs(val_set, val_img_dir, val_lbl_dir)


print("\n===== Step 4: data.yaml 생성 =====")
yaml_path = os.path.join(BASE, "data.yaml")

data_yaml = {
    "train": "train/images",
    "val": "val/images",
    "nc": 1,
    "names": ["box"]
}

with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print(f"data.yaml 생성됨 → {yaml_path}")

print("\n===== ✔ 전체 데이터셋 준비 완료 =====\n")




===== Step 1: JSON 파일 삭제 =====
삭제됨 → color_output_2_000636.json
삭제됨 → frame_000590.json
삭제됨 → frame_000810.json
삭제됨 → frame_1000220.json
삭제됨 → color_output_1_000668.json
삭제됨 → frame_1000730.json
삭제됨 → color_output_1_000726.json
삭제됨 → color_output_2_000633.json
삭제됨 → color_output_1_000661.json
삭제됨 → color_output_1_001351.json
삭제됨 → color_output_1_003438.json
삭제됨 → color_output_1_000772.json
삭제됨 → color_output_1_001629.json
삭제됨 → color_output_1_000677.json
삭제됨 → color_output_1_001545.json
삭제됨 → color_output_1_002600.json
삭제됨 → color_output_1_000377.json
삭제됨 → color_output_1_001512.json
삭제됨 → color_output_2_000746.json
삭제됨 → color_output_1_002007.json
삭제됨 → color_output_1_001157.json
삭제됨 → color_output_1_000721.json
삭제됨 → color_output_1_002012.json
삭제됨 → color_output_1_002608.json
삭제됨 → color_output_1_001620.json
삭제됨 → color_output_1_001358.json
삭제됨 → frame_2000250.json
삭제됨 → color_output_1_002417.json
삭제됨 → color_output_1_001511.json
삭제됨 → color_output_1_003342.json
삭제됨 → color_output_1

In [9]:
from ultralytics import YOLO

# ----------------------------------------------------
# 1) YOLO11S-OBB 모델 로드
# ----------------------------------------------------
model = YOLO("yolo11s-obb.pt")   # OBB 전용 pretrained 모델

# ----------------------------------------------------
# 2) DATA.YAML 경로
# ----------------------------------------------------
DATA_PATH = "/home/dw/ws_job_msislab/amr_project/for_training/20251218_obb/data.yaml"

# ----------------------------------------------------
# 3) 학습 실행
# ----------------------------------------------------
model.train(
    # ---------- 필수 ----------
    data=DATA_PATH,
    epochs=500,                # 스모크 테스트
    imgsz=640,
    batch=4,
    device=0,
    workers=2,

    # ---------- 실험 관리 ----------
    project="runs/obb",      # runs/obb 아래로 정리
    name="smoke_test_v2",    # 실험 이름
    exist_ok=True,           # 같은 이름 덮어쓰기 허용
    seed=25,                 # 재현성 확보
    deterministic=True,

    # ---------- Optimizer / LR ----------
    optimizer="AdamW",       # OBB에서 비교적 안정적
    lr0=1e-3,                # 초기 learning rate
    lrf=0.01,                # 최종 LR 비율
    cos_lr=True,             # cosine scheduler

    # ---------- 학습 안정성 ----------
    patience=15,             # early stopping (스모크라 사실상 의미 없음)
    warmup_epochs=1,
    amp=True,                # mixed precision (GPU 메모리 절약)

    # ---------- 데이터 로딩 ----------
    cache="ram",             # 데이터 적으면 RAM 캐시 추천
    rect=False,              # OBB는 보통 False 권장

    # ---------- Augmentation (OBB 기준 보수적으로) ----------
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=5.0,             # 회전은 OBB 특성상 소량만
    translate=0.1,
    scale=0.5,
    shear=2.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=0.5,
    mixup=0.0,

    # ---------- 저장 / 로그 ----------
    save=True,
    save_period=0,           # epoch마다 weight 저장
    plots=True,              # loss, metrics plot 생성
    verbose=True,
)

print("\n🎉 Training Finished! Check the 'runs/obb/smoke_test_v1' folder.")



New https://pypi.org/project/ultralytics/8.3.240 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.239 🚀 Python-3.10.12 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/home/dw/ws_job_msislab/amr_project/for_training/20251218_obb/data.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-obb.pt, momentum=0.937, mosaic=0.5, mu

In [ ]:
from ultralytics import YOLO
import os
import glob
import cv2

# -------------------------------------------------------
# 1) 모델 로드
# -------------------------------------------------------
MODEL_PATH = "/home/dw/ws_job_msislab/amr_project/src/runs/obb/smoke_test_v1/weights/best.pt"
model = YOLO(MODEL_PATH)

# -------------------------------------------------------
# 2) 루트 이미지 폴더
# -------------------------------------------------------
ROOT = "/home/dw/ws_job_msislab/amr_project/data/image_data/for_yolo_obj/20251210"

# 서브폴더까지 모든 이미지 검색
img_list = []
for ext in ["*.jpg", "*.png", "*.jpeg", "*.JPG", "*.PNG"]:
    img_list.extend(glob.glob(f"{ROOT}/**/{ext}", recursive=True))

print(f"[INFO] 총 {len(img_list)}장의 이미지를 inference 합니다.")

# -------------------------------------------------------
# 3) 출력 폴더
# -------------------------------------------------------
OUT_DIR = os.path.join(ROOT, "obb_viz")
os.makedirs(OUT_DIR, exist_ok=True)

# -------------------------------------------------------
# 4) inference + 시각화 저장
# -------------------------------------------------------
for img_path in img_list:
    file_name = os.path.basename(img_path)
    out_path = os.path.join(OUT_DIR, file_name)

    results = model(img_path)
    plotted = results[0].plot()

    cv2.imwrite(out_path, plotted)
    print(f"[OK] {out_path} 저장됨")

print("\n🎉 완료! obb_viz 폴더에서 시각화 결과 확인!")


### obb 각도랑 뭐시기

In [ ]:
from ultralytics import YOLO
import os
import glob
import cv2
import numpy as np
import math

# -------------------------------------------------------
# 1) 모델 로드
# -------------------------------------------------------
MODEL_PATH = "/home/dw/ws_job_msislab/amr_project/src/runs/obb/smoke_test_v1/weights/best.pt"
model = YOLO(MODEL_PATH)

# -------------------------------------------------------
# 2) 루트 이미지 폴더
# -------------------------------------------------------
ROOT = "/home/dw/ws_job_msislab/amr_project/data/image_data/for_yolo_obj/20251210"

img_list = []
for ext in ["*.jpg", "*.png", "*.jpeg", "*.JPG", "*.PNG"]:
    img_list.extend(glob.glob(f"{ROOT}/**/{ext}", recursive=True))

print(f"[INFO] 총 {len(img_list)}장의 이미지를 inference 합니다.")

# -------------------------------------------------------
# 3) 출력 폴더
# -------------------------------------------------------
OUT_DIR = os.path.join(ROOT, "obb_viz")
os.makedirs(OUT_DIR, exist_ok=True)

# -------------------------------------------------------
# 4) inference + 커스텀 시각화
# -------------------------------------------------------
for img_path in img_list:
    img = cv2.imread(img_path)
    if img is None:
        continue

    results = model(img)
    r = results[0]

    if r.obb is None:
        continue

    boxes = r.obb.xyxyxyxy.cpu().numpy()   # (N, 4, 2)
    confs = r.obb.conf.cpu().numpy()

    for box, conf in zip(boxes, confs):
        # -------------------------------
        # OBB 좌표
        # -------------------------------
        pts_int = box.astype(np.int32)     # 시각화용
        pts = box.astype(np.float32)       # 계산용

        # -------------------------------
        # OBB 외곽선
        # -------------------------------
        cv2.polylines(img, [pts_int], True, (255, 0, 0), 2)

        # -------------------------------
        # 중심점
        # -------------------------------
        cx, cy = pts.mean(axis=0)
        cx_i, cy_i = int(cx), int(cy)

        # -------------------------------
        # 박스 방향 벡터 (장축 / 단축)
        # -------------------------------
        p0, p1, p2, p3 = pts

        v_long = p1 - p0
        v_short = p2 - p1

        len_long = np.linalg.norm(v_long)
        len_short = np.linalg.norm(v_short)

        if len_long < 1 or len_short < 1:
            continue

        v_long /= len_long
        v_short /= len_short

        # -------------------------------------------------
        # 🔥 장축 벡터 방향 통일: 항상 "위쪽(Y-)"을 향하게
        #    (부호 튐 방지 핵심)
        # -------------------------------------------------
        if v_long[1] > 0:      # 아래쪽(+y)을 향하면
            v_long = -v_long   # 위쪽(-y)으로 뒤집기

        # -------------------------------
        # 박스 전체 십자가 (회전 포함)
        # -------------------------------
        long_p1 = (
            int(cx - v_long[0] * len_long / 2),
            int(cy - v_long[1] * len_long / 2),
        )
        long_p2 = (
            int(cx + v_long[0] * len_long / 2),
            int(cy + v_long[1] * len_long / 2),
        )

        short_p1 = (
            int(cx - v_short[0] * len_short / 2),
            int(cy - v_short[1] * len_short / 2),
        )
        short_p2 = (
            int(cx + v_short[0] * len_short / 2),
            int(cy + v_short[1] * len_short / 2),
        )

        cv2.line(img, long_p1, long_p2, (0, 255, 255), 2)
        cv2.line(img, short_p1, short_p2, (0, 255, 255), 2)

        # -------------------------------
        # ✅ 회전 각도 (수직 기준)
        #   0°  : 정자세 (세로)
        #   +   : 오른쪽 기울어짐
        #   -   : 왼쪽 기울어짐
        # -------------------------------
        # [중요] 여기만 "부호 확정" 로직을 추가함 (나머진 그대로)
        v_ref = np.array([0.0, -1.0], dtype=np.float32)  # 수직(위쪽) 기준

        # 1) 각도 크기 (0~180) 중 실제론 0~90 근처로 나옴
        cos_theta = float(np.clip(np.dot(v_long, v_ref), -1.0, 1.0))
        angle_abs = math.degrees(math.acos(cos_theta))

        # 2) 부호: 외적 z성분으로 좌/우 결정
        #   cross > 0  => 오른쪽 기울(+) / cross < 0 => 왼쪽 기울(-)
        cross = v_ref[0] * v_long[1] - v_ref[1] * v_long[0]
        angle = angle_abs if cross > 0 else -angle_abs

        # -------------------------------
        # 각도 텍스트
        # -------------------------------
        label = f"{angle:.1f} deg"
        cv2.putText(
            img,
            label,
            (cx_i - 40, cy_i - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 0, 255),
            2,
        )

    # -------------------------------
    # 결과 저장
    # -------------------------------
    out_path = os.path.join(OUT_DIR, os.path.basename(img_path))
    cv2.imwrite(out_path, img)
    print(f"[OK] {out_path}")

print("\n🎉 완료! 수직 기준 각도(+/-) + 십자가 시각화 결과 확인")


### obb시각화 2번째

In [ ]:
from ultralytics import YOLO
import os
import glob
import cv2
import numpy as np
import math

# -------------------------------------------------------
# 1) 모델 로드
# -------------------------------------------------------
MODEL_PATH = "/home/dw/ws_job_msislab/amr_project/src/runs/obb/smoke_test_v1/weights/best.pt"
model = YOLO(MODEL_PATH)

# -------------------------------------------------------
# 2) 루트 이미지 폴더
# -------------------------------------------------------
ROOT = "/home/dw/ws_job_msislab/amr_project/data/image_data/for_yolo_obj/20251210"

img_list = []
for ext in ["*.jpg", "*.png", "*.jpeg", "*.JPG", "*.PNG"]:
    img_list.extend(glob.glob(f"{ROOT}/**/{ext}", recursive=True))

print(f"[INFO] 총 {len(img_list)}장의 이미지를 inference 합니다.")

# -------------------------------------------------------
# 3) 출력 폴더
# -------------------------------------------------------
OUT_DIR = os.path.join(ROOT, "obb_viz")
os.makedirs(OUT_DIR, exist_ok=True)

# -------------------------------------------------------
# 4) inference + 커스텀 시각화
# -------------------------------------------------------
for img_path in img_list:
    img = cv2.imread(img_path)
    if img is None:
        continue

    results = model(img)
    r = results[0]

    if r.obb is None:
        continue

    boxes = r.obb.xyxyxyxy.cpu().numpy()   # (N, 4, 2)
    confs = r.obb.conf.cpu().numpy()

    for box, conf in zip(boxes, confs):
        # -------------------------------
        # OBB 좌표
        # -------------------------------
        pts_int = box.astype(np.int32)     # 시각화용
        pts = box.astype(np.float32)       # 계산용

        # -------------------------------
        # OBB 외곽선
        # -------------------------------
        cv2.polylines(img, [pts_int], True, (255, 0, 0), 2)

        # -------------------------------
        # 중심점
        # -------------------------------
        cx, cy = pts.mean(axis=0)
        cx_i, cy_i = int(cx), int(cy)

        # -------------------------------
        # 박스 방향 벡터 (장축 / 단축)  ✅ 여기 핵심 수정
        # -------------------------------
        p0, p1, p2, p3 = pts

        e01 = p1 - p0
        e12 = p2 - p1

        len01 = np.linalg.norm(e01)
        len12 = np.linalg.norm(e12)

        if len01 < 1 or len12 < 1:
            continue

        # ✅ 더 긴 변을 장축(long)으로 선택
        if len01 >= len12:
            v_long = e01
            v_short = e12
            len_long = len01
            len_short = len12
        else:
            v_long = e12
            v_short = e01
            len_long = len12
            len_short = len01

        # 정규화
        v_long /= len_long
        v_short /= len_short

        # -------------------------------------------------
        # 🔥 장축 벡터 방향 통일: 항상 "위쪽(Y-)"을 향하게
        # -------------------------------------------------
        if v_long[1] > 0:      # 아래쪽(+y)을 향하면
            v_long = -v_long   # 위쪽(-y)으로 뒤집기

        # -------------------------------
        # 박스 전체 십자가 (회전 포함)
        # -------------------------------
        long_p1 = (
            int(cx - v_long[0] * len_long / 2),
            int(cy - v_long[1] * len_long / 2),
        )
        long_p2 = (
            int(cx + v_long[0] * len_long / 2),
            int(cy + v_long[1] * len_long / 2),
        )

        short_p1 = (
            int(cx - v_short[0] * len_short / 2),
            int(cy - v_short[1] * len_short / 2),
        )
        short_p2 = (
            int(cx + v_short[0] * len_short / 2),
            int(cy + v_short[1] * len_short / 2),
        )

        cv2.line(img, long_p1, long_p2, (0, 255, 255), 2)
        cv2.line(img, short_p1, short_p2, (0, 255, 255), 2)

        # -------------------------------
        # ✅ 회전 각도 (수직 기준)
        #   0°  : 정자세 (세로)
        #   +   : 오른쪽 기울어짐
        #   -   : 왼쪽 기울어짐
        # -------------------------------
        angle = math.degrees(math.atan2(v_long[0], -v_long[1]))

        # -------------------------------
        # 각도 텍스트
        # -------------------------------
        label = f"{angle:.1f} deg"
        cv2.putText(
            img,
            label,
            (cx_i - 40, cy_i - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 0, 255),
            2,
        )

    # -------------------------------
    # 결과 저장
    # -------------------------------
    out_path = os.path.join(OUT_DIR, os.path.basename(img_path))
    cv2.imwrite(out_path, img)
    print(f"[OK] {out_path}")

print("\n🎉 완료! 수직 기준 각도(+/-) + 십자가 시각화 결과 확인")


In [ ]:
from ultralytics import YOLO
import pyrealsense2 as rs
import numpy as np
import cv2
import math
import time

# -------------------------------------------------------
# 0) 설정
# -------------------------------------------------------
MODEL_PATH = "/home/dw/ws_job_msislab/amr_project/src/runs/obb/smoke_test_v1/weights/best.pt"
CONF_TH = 0.25
IMGSZ = 640
DEVICE = 0  # GPU 없으면 "cpu"

COLOR_W, COLOR_H, FPS = 640, 480, 30
DEPTH_W, DEPTH_H = 640, 480

# depth 샘플링: 중심 주변에서 N x N 픽셀을 뽑아 median
DEPTH_PATCH = 9          # 홀수 권장 (예: 5, 7, 9, 11)
DEPTH_MIN_M = 0.10       # 유효 depth 최소
DEPTH_MAX_M = 2.50       # 유효 depth 최대

# -------------------------------------------------------
# 1) YOLO 로드
# -------------------------------------------------------
model = YOLO(MODEL_PATH)

# -------------------------------------------------------
# 2) RealSense 파이프라인
# -------------------------------------------------------
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, COLOR_W, COLOR_H, rs.format.bgr8, FPS)
config.enable_stream(rs.stream.depth, DEPTH_W, DEPTH_H, rs.format.z16, FPS)

profile = pipeline.start(config)

# color에 depth 정렬
align = rs.align(rs.stream.color)

# intrinsics (color stream 기준)
color_stream = profile.get_stream(rs.stream.color).as_video_stream_profile()
intr = color_stream.get_intrinsics()
fx, fy, ppx, ppy = intr.fx, intr.fy, intr.ppx, intr.ppy

print("[INFO] RealSense started")
print(f"[INFO] intrinsics: fx={fx:.2f}, fy={fy:.2f}, ppx={ppx:.2f}, ppy={ppy:.2f}")
print("[INFO] Press 'q' to quit")

def median_depth_in_patch(depth_frame, u, v, patch=9):
    """
    (u,v) 중심 주변 patch x patch 영역의 depth(m) median 반환.
    depth_frame은 align된 depth_frame이어야 함.
    """
    half = patch // 2
    h, w = depth_frame.get_height(), depth_frame.get_width()

    u0 = max(0, u - half)
    u1 = min(w - 1, u + half)
    v0 = max(0, v - half)
    v1 = min(h - 1, v + half)

    vals = []
    for yy in range(v0, v1 + 1):
        for xx in range(u0, u1 + 1):
            d = depth_frame.get_distance(xx, yy)  # meters
            if DEPTH_MIN_M <= d <= DEPTH_MAX_M:
                vals.append(d)

    if not vals:
        return None
    return float(np.median(vals))

def compute_obb_axes_and_angle(pts):
    """
    pts: (4,2) float32, OBB 4점(시계방향)
    return:
      cx,cy (float), v_long(unit), v_short(unit),
      len_long(px), len_short(px),
      angle_deg (vertical 기준 0°, 오른쪽 +, 왼쪽 -)
    """
    cx, cy = pts.mean(axis=0)

    p0, p1, p2, p3 = pts

    e01 = p1 - p0
    e12 = p2 - p1

    len01 = np.linalg.norm(e01)
    len12 = np.linalg.norm(e12)
    if len01 < 1e-6 or len12 < 1e-6:
        return None

    # 더 긴 변을 장축으로 선택 (너 케이스 70도 튐 해결)
    if len01 >= len12:
        v_long = e01
        v_short = e12
        len_long, len_short = len01, len12
    else:
        v_long = e12
        v_short = e01
        len_long, len_short = len12, len01

    v_long = v_long / (len_long + 1e-12)
    v_short = v_short / (len_short + 1e-12)

    # 장축 방향 통일: 항상 위쪽(-y) 향하도록 (부호 안정화)
    if v_long[1] > 0:
        v_long = -v_long

    # 세로 기준 각도:
    # 0° = 위쪽(-y)과 일치
    # 오른쪽 기울면 +, 왼쪽 기울면 -
    angle_deg = math.degrees(math.atan2(v_long[0], -v_long[1]))

    return cx, cy, v_long, v_short, float(len_long), float(len_short), float(angle_deg)

try:
    while True:
        frames = pipeline.wait_for_frames()
        frames = align.process(frames)

        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        color = np.asanyarray(color_frame.get_data())

        # ---------------------------
        # YOLO OBB inference
        # ---------------------------
        results = model.predict(
            source=color,
            imgsz=IMGSZ,
            conf=CONF_TH,
            device=DEVICE,
            verbose=False
        )
        r = results[0]

        if r.obb is not None and len(r.obb) > 0:
            boxes = r.obb.xyxyxyxy.cpu().numpy()  # (N,4,2)
            confs = r.obb.conf.cpu().numpy()

            # 가장 높은 conf 1개만 선택(테스트용)
            best_i = int(np.argmax(confs))
            box = boxes[best_i]
            conf = float(confs[best_i])

            pts_i = box.astype(np.int32)
            pts = box.astype(np.float32)

            # OBB 외곽선
            cv2.polylines(color, [pts_i], True, (255, 0, 0), 2)

            axes = compute_obb_axes_and_angle(pts)
            if axes is not None:
                cx, cy, v_long, v_short, len_long, len_short, angle_deg = axes
                cx_i, cy_i = int(cx), int(cy)

                # 박스 전체 십자가
                long_p1 = (int(cx - v_long[0]*len_long/2), int(cy - v_long[1]*len_long/2))
                long_p2 = (int(cx + v_long[0]*len_long/2), int(cy + v_long[1]*len_long/2))
                short_p1 = (int(cx - v_short[0]*len_short/2), int(cy - v_short[1]*len_short/2))
                short_p2 = (int(cx + v_short[0]*len_short/2), int(cy + v_short[1]*len_short/2))
                cv2.line(color, long_p1, long_p2, (0, 255, 255), 2)
                cv2.line(color, short_p1, short_p2, (0, 255, 255), 2)

                # depth (중심 주변 median)
                Z = median_depth_in_patch(depth_frame, cx_i, cy_i, patch=DEPTH_PATCH)

                if Z is not None:
                    # deproject: pixel + depth -> 3D (camera frame)
                    X, Y, Zm = rs.rs2_deproject_pixel_to_point(intr, [cx_i, cy_i], Z)
                    # rs2_deproject_pixel_to_point returns [X,Y,Z] in meters
                    dist = math.sqrt(X*X + Y*Y + Zm*Zm)

                    text1 = f"conf={conf:.2f}  angle={angle_deg:+.1f} deg"
                    text2 = f"cam XYZ=({X:+.3f},{Y:+.3f},{Zm:+.3f}) m  dist={dist:.3f} m"
                else:
                    text1 = f"conf={conf:.2f}  angle={angle_deg:+.1f} deg"
                    text2 = "depth: N/A (try adjust DEPTH_PATCH / object surface)"

                cv2.putText(color, text1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(color, text2, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        cv2.imshow("YOLO11-OBB + Depth (Camera frame)", color)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()
    print("[INFO] stopped")
